In [195]:
from opendbc.car import structs
from opendbc.car.honda.values import CAR

TEST_PLATFORMS = set(CAR)

print(f"Test set contains {len(TEST_PLATFORMS)} qualifying platforms:")
for platform in TEST_PLATFORMS:
  print(f"{platform}")

Test set contains 23 qualifying platforms:
HONDA_PILOT
HONDA_CIVIC_2022
HONDA_FREED
HONDA_ODYSSEY_CHN
HONDA_CRV
HONDA_ODYSSEY
HONDA_HRV
HONDA_CRV_EU
ACURA_ILX
ACURA_RDX_3G
HONDA_CIVIC_BOSCH_DIESEL
HONDA_FIT
HONDA_HRV_3G
HONDA_INSIGHT
HONDA_CRV_HYBRID
HONDA_CIVIC_BOSCH
ACURA_RDX
HONDA_PILOT_4G
HONDA_ACCORD
HONDA_CRV_5G
HONDA_E
HONDA_RIDGELINE
HONDA_CIVIC


In [230]:
import random

from openpilot.tools.lib.logreader import LogReader
from openpilot.tools.lib.comma_car_segments import get_comma_car_segments_database

database = get_comma_car_segments_database()
TEST_SEGMENTS = []

MAX_SEGS_PER_PLATFORM = 100  # TODO: Increase this to search more segments

print("Collecting segments from commaCarSegments dataset:\n")
for platform in TEST_PLATFORMS:
  # assert(platform in database)
  if platform not in database:
    print(f"Skipping platform: {platform}, no data available")
    continue
  
  all_segments = database[platform]

  NUM_SEGMENTS = min(len(all_segments), MAX_SEGS_PER_PLATFORM)

  print(f"Got {len(all_segments)} segments for platform {platform}, sampling {NUM_SEGMENTS} segments")

  segment_subset = random.sample(all_segments, NUM_SEGMENTS)
  for segment in segment_subset:
    TEST_SEGMENTS.append([platform, segment])

print(f"\nSampled {len(TEST_SEGMENTS)} segments from commaCarSegments")

# For new car port development, not yet upstream
EXTRA_SEGMENTS = [
  ["ACURA_MDX_4G", "f9c43864cf057d05|2024-01-15--23-01-20/0"],  # CAN-FD
  ["ACURA_INTEGRA", "3f8ae015ce70365f/00000003--a22590d0e4/0"],
  ["HONDA_CITY", "56b2cf1dacdcd033/00000048--3267801001/0"],
  # ["HONDA_ODYSSEY_5G_MMR", "558ac87ada6a88cd/00000036--7d59e2af08/0"],  # no rlogs
  ["ACURA_MDX_4G_MMR", "ad9840558640c31d/0000001a--d6cd4871c2/0"],  # CAN-FD
  ["ACURA_RDX_3G_MMR", "0416c2843c0bfc91/0000001d--59c7744d87/0"],
  ["HONDA_NBOX", "414af83891dbf72c/00000006--51fa6d99cd/0"],
]

TEST_SEGMENTS.extend(EXTRA_SEGMENTS)

print(f"Added {len(EXTRA_SEGMENTS)} manual segments")



Got 3239 segments for platform HONDA_PILOT, sampling 100 segments
Got 1341 segments for platform HONDA_CIVIC_2022, sampling 100 segments
Got 50 segments for platform HONDA_FREED, sampling 50 segments
Skipping platform: HONDA_ODYSSEY_CHN, no data available
Got 156 segments for platform HONDA_CRV, sampling 100 segments
Got 19 segments for platform HONDA_ODYSSEY, sampling 19 segments
Got 150 segments for platform HONDA_HRV, sampling 100 segments
Skipping platform: HONDA_CRV_EU, no data available
Got 4 segments for platform ACURA_ILX, sampling 4 segments
Got 747 segments for platform ACURA_RDX_3G, sampling 100 segments
Skipping platform: HONDA_CIVIC_BOSCH_DIESEL, no data available
Got 282 segments for platform HONDA_FIT, sampling 100 segments
Got 60 segments for platform HONDA_HRV_3G, sampling 60 segments
Got 661 segments for platform HONDA_INSIGHT, sampling 100 segments
Got 359 segments for platform HONDA_CRV_HYBRID, sampling 100 segments
Got 12 segments for platform HONDA_CIVIC_BOSCH, s

In [233]:
import copy
from math import isclose
import matplotlib.pyplot as plt
import numpy as np

from opendbc.can.parser import CANParser
from opendbc.car.honda.values import DBC
from opendbc.car.honda.hondacan import CanBus

from openpilot.selfdrive.pandad import can_capnp_to_list
from openpilot.tools.lib.logreader import LogReader

door_address = 1029
position_addresses = [392, 401, 419]
mt_address = 477
all_addresses = [*position_addresses, mt_address, door_address]
buses = [0, 1]

car_results = {}
for segment in TEST_SEGMENTS:
  car_results[segment[0]] = {}
  for address in all_addresses:
    car_results[segment[0]][address] = 0

for segment in TEST_SEGMENTS:
  print(f"Analyzing segment {segment[1]:<44} for {segment[0]}")
  lr = LogReader(segment[1])
  CP = lr.first("carParams")
  if CP is None:
    continue


  segment_counts = {}
  gear_activity = {}
  for address in all_addresses:
    segment_counts[address] = 0
    gear_activity[address] = False
    
  can_msgs = [msg for msg in lr if msg.which() == "can"]
  for can_msg in can_msgs:
    for packet in can_msg.can:
      for address in all_addresses:
        if packet.src in buses and packet.address == address:
          if address == 392:
            assert len(packet.dat) == 6
            segment_counts[392] += 1
            if packet.dat[0] & 0x7F != 0:
              gear_activity[address] = True
          if address == 401:
            assert len(packet.dat) == 8
            segment_counts[401] += 1
            if packet.dat[3] & 0xF != 0:
              gear_activity[address] = True
          if address == 419:
            assert len(packet.dat) == 8
            segment_counts[419] += 1
            if packet.dat[3] & 0x7F != 0:
              gear_activity[address] = True
          if address == 477:
            # Only used for MT, but often present on non-MT cars, not as easy to detect gear activity
            assert len(packet.dat) == 8
            segment_counts[477] += 1
          if address == 1029:
            assert len(packet.dat) == 8
            segment_counts[1029] += 1

  # Make sure gear position activity was found, or the MT message is present
  print(segment_counts)
    
  gear_position_found = False
  for address in position_addresses:
    if segment_counts[address] > 0:
      frequency = round(segment_counts[address] / 60)
      if gear_activity[address]:
        print(f"Found shifter position: GEARBOX_{address} at {frequency}Hz")
        assert(address == 392 and isclose(frequency, 100, abs_tol=20) or \
               address == 401 and isclose(frequency, 100, abs_tol=20) or \
               address == 419 and isclose(frequency, 50, abs_tol=10))               
        car_results[segment[0]][address] += 1
        gear_position_found = True

  if not gear_position_found:
    address = mt_address
    frequency = round(segment_counts[address] / 60)
    print(f"Found manual trans car: GEARBOX_477 at {frequency}Hz")
    assert isclose(frequency, 50, abs_tol=5)
    car_results[segment[0]][address] += 1

  if segment_counts[door_address] > 0:
    frequency = round(segment_counts[door_address] / 60)
    print(f"Found door message: DOORS_OPEN at {frequency}Hz")
    assert isclose(frequency, 3, abs_tol=1)
    car_results[segment[0]][door_address] += 1

print(f"Analysis finished")


Analyzing segment c31ea510a692b5c3/000000eb--0d78ddcfe0/5/s    for HONDA_PILOT
{392: 0, 401: 0, 419: 3000, 477: 0, 1029: 200}
Found shifter position: GEARBOX_419 at 50Hz
Found door message: DOORS_OPEN at 3Hz
Analyzing segment 9dae484044b3cfe8/000003e6--79f1b775d3/15/s   for HONDA_PILOT
{392: 0, 401: 0, 419: 3000, 477: 0, 1029: 199}
Found shifter position: GEARBOX_419 at 50Hz
Found door message: DOORS_OPEN at 3Hz
Analyzing segment 50b9db073fd0ea07/000005d1--1be0173955/27/s   for HONDA_PILOT
{392: 0, 401: 0, 419: 2999, 477: 0, 1029: 199}
Found shifter position: GEARBOX_419 at 50Hz
Found door message: DOORS_OPEN at 3Hz
Analyzing segment 6868a58803419504/00000023--cbeeef8a29/3/s    for HONDA_PILOT
{392: 0, 401: 0, 419: 3000, 477: 0, 1029: 199}
Found shifter position: GEARBOX_419 at 50Hz
Found door message: DOORS_OPEN at 3Hz
Analyzing segment 9098b6df85df4bab/00000115--f3134c11a7/63/s   for HONDA_PILOT
{392: 0, 401: 0, 419: 3000, 477: 0, 1029: 199}
Found shifter position: GEARBOX_419 at 50H

In [236]:
print(f"Gear messages by platform      0x188 0x191 0x1A3 0x1DD 0x405")
print(f"------------------------------ ----- ----- ----- ----- -----")
for platform in sorted(car_results):
  print(f"{platform:30} {car_results[platform][392]:5} {car_results[platform][401]:5} {car_results[platform][419]:5} {car_results[platform][477]:5} {car_results[platform][1029]:5}")


Gear messages by platform      0x188 0x191 0x1A3 0x1DD 0x405
------------------------------ ----- ----- ----- ----- -----
ACURA_ILX                          0     0     4     0     4
ACURA_INTEGRA                      0     1     0     0     0
ACURA_MDX_4G                       0     0     1     0     0
ACURA_MDX_4G_MMR                   0     0     1     0     0
ACURA_RDX                        100     0     0     0   100
ACURA_RDX_3G                       0     0   100     0     0
ACURA_RDX_3G_MMR                   0     0     1     0     0
HONDA_ACCORD                       0    16     0     0     0
HONDA_CITY                         0     1     0     0     0
HONDA_CIVIC                        0   100     0     0   100
HONDA_CIVIC_2022                   0    72    22     6     0
HONDA_CIVIC_BOSCH                  0    12     0     0     5
HONDA_CRV                          0   100     0     0   100
HONDA_CRV_5G                       0   100     0     0   100
HONDA_CRV_HYBRID        